In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
df_info = df_info[(df_info.id_direction==1) & (df_info.id_location==1)]
df_info = df_info[df_info.id_person < 30]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_time = 500#int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)


C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# make data generator
def gen_data(batch_size,df_info,id_num,len_num):
    nested_gen = gen_csi(df_info,id_num,len_num)
    for i in range(len(np.unique(df_info.id.values))):
        list_data = []
        list_label = []
        for j in range(batch_size):
            data_read, label_read = next(nested_gen)
            list_data.append(data_read)
            list_label.append(label_read)
        yield(np.array(list_data),np.array(list_label))
def gen_csi(df_info,id_num,len_num):
    for file in np.unique(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        data1_abs = np.abs(data1_norm)
        data1_ph = np.angle(data1_norm)
        
        data1_diff = np.abs(data1[:500])
        
        # differentiation
        #data1_diff = np.diff(data1_abs,axis=0)
        
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]

        x_input = data1_pad.reshape([len_num,30,6]).astype('float32')
        y_input = tf.keras.utils.to_categorical(data1_y, no_classes)
        
        yield(x_input / max_value,y_input)

In [3]:
#models
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [4]:
#Train Test Split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,random_state=10)
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

gen_train = gen_data(10,df_train,no_classes,csi_time)
gen_test = gen_data(10,df_test,no_classes,csi_time)

#trining
simple_cnn_model = simple_cnn(input_shape)
simple_cnn_model.fit_generator(gen_train,steps_per_epoch=1,epochs=30,
                              validation_data=gen_test,validation_steps=1)

Epoch 1/30
1/1 [==============================] - 27s 27s/step - loss: 2.4041 - acc: 0.0000e+00 - val_loss: 1.6118 - val_acc: 0.9000
Epoch 2/30
1/1 [==============================] - 11s 11s/step - loss: 14.5063 - acc: 0.1000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 12s 12s/step - loss: 16.1181 - acc: 0.0000e+00 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 9s 9s/step - loss: 16.1181 - acc: 0.0000e+00 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/30


StopIteration: 

In [ ]:
# get accuracy on single data
ex_train = next(gen_train)
train_loss, train_accuracy = simple_cnn_model.evaluate(
    ex_train[0], ex_train[1], verbose=0)
print('Train data loss:', train_loss)
print('Train data accuracy:', train_accuracy)

In [ ]:
#Train Test Split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,random_state=10)
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

gen_train = gen_data(10,df_train,no_classes,csi_time)
gen_test = gen_data(10,df_test,no_classes,csi_time)

In [ ]:
gen1 = gen_data(10,df_info,no_classes,csi_time)

In [ ]:
a,b = next(gen1)

In [ ]:
a.shape,b.shape

In [ ]:
b